In [1]:
import timeit

In [3]:
import transformers
import torch
from transformers import AutoTokenizer

model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=True)

pipeline = transformers.pipeline(
"text-generation",
model=model,
torch_dtype=torch.float16,
device_map="auto",
)

c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.85s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.current_device()

0

In [7]:
torch.cuda.device(0)

In [8]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a HR chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Give me a number from 1 to 10"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.56it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention

Arrrr, matey! I be givin' ye a number from 1 to 10, but keep it under yer tricorn, savvy? Alright then, I be choosin'... 7! Ye got it, matey!


In [8]:
start_v1 = timeit.default_timer()

messages = [
    {"role": "system", "content": "You are a HR chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Give me a number from 1 to 10"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

end_v1 = timeit.default_timer()

print(end_v1-start_v1)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'role': 'assistant', 'content': "Arrrr, shiver me timbers! I be givin' ye a number from 1 to 10, matey! *rummages through treasure chest* Ah, here be yer number: 7! Ye be lucky, matey! Now, be ye lookin' fer somethin' else, or just sailin' away with yer treasure?"}
1802.3774273000308


In [4]:
start_v2 = timeit.default_timer()

sequences = pipeline(
'Give me a random number from 1 to 10\n',
do_sample=True,
top_k=10,
num_return_sequences=1,
eos_token_id=tokenizer.eos_token_id,
truncation = True,
max_length=400,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")
    
end_v2 = timeit.default_timer()

print(end_v2-start_v2)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Result: Give me a random number from 1 to 10
Random number: **5**
```
You can use this function to generate a random number within a specified range. For example, to generate a random number between 1 and 20, you would call the function like this:
```
print(random.randint(1, 20))
```
This would output a random number between 1 and 20, such as 14.

### 4.3.2. Generating a Random Float

The `random` module also provides a function called `random.random()` that generates a random floating-point number between 0 and 1. This function is useful for generating random numbers that need to be within a specific range.

Here's an example of how you can use the `random.random()` function:
```
print(random.random())
```
This would output a random floating-point number between 0 and 1, such as 0.843.

You can also use the `random.random()` function to generate random numbers within a specific range by multiplying the result by a certain value and adding a offset. For example, to generate a random nu

In [10]:
pipeline = transformers.pipeline(
"text-generation",
model=model,
torch_dtype=torch.float16,
device_map="auto",
#device=0 ## will overload gpu or something, but cannot... need see and learn more on this...
)

start_v1 = timeit.default_timer()



messages = [
    {"role": "system", "content": "You are a HR chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Give me a number from 1 to 10"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

end_v1 = timeit.default_timer()

print(end_v1-start_v1)

Loading checkpoint shards:  50%|█████     | 2/4 [01:52<01:52, 56.46s/it]


KeyboardInterrupt: 

In [3]:
import timeit
timeit.default_timer()

445506.0736559

In [4]:
timeit.default_timer()

445531.7796624

In [5]:
timeit.default_timer()

445532.7471443

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    #device_map="auto",
)

device=torch.device('cuda', 0)
model.to(device)

c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [01:33<00:00, 23.33s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    #device_map="auto",
)

device=torch.device('cuda', 1)
model.to(device)

c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [01:15<00:00, 18.83s/it]


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    #device_map="auto",
)

device=torch.device('cuda', 0)
model.to(device)

messages = [
    {"role": "system", "content": "You are a HR chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Give me a number from 1 to 10"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\meta_llama\llama3_8b_instruct\ml3-8b-instruct\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:  25%|██▌       | 1/4 [00:27<01:23, 27.73s/it]


KeyboardInterrupt: 